In [1]:
import numpy as np
import pandas as pd

import xarray as xr
import matplotlib.pyplot as plt

import os 
import time

from renewable_data_load import *


# Configure dask for parallel execution
#import dask
#dask.config.set(scheduler='threads', num_workers=10)  # Adjust num_workers to your CPU cores

/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/intake_esm/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [2]:
# ds_doy.isel(x=110,y=230).plot(x='year',y='dayofyear', cmap='viridis')
# drought_ds.isel(x=110,y=230).plot(x='year',y='dayofyear', cmap='viridis')

In [2]:
resource = "pv"
module = "utility"

reference_gwl = 0.8
future_gwl = 2.0

simulations = ["mpi-esm1-2-hr", "miroc6", "taiesm1", 'ec-earth3',]
#simulations = ['ec-earth3']

simulation = "mpi-esm1-2-hr"


In [3]:

# domain = "d02"
# variable = "cf"
# frequency = "day"

# threshold_file = f"../data/thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
# drought_threshold_ds = xr.open_dataset(threshold_file)
# ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
# ren_ds = ren_ds.convert_calendar("noleap")

# # reshape array
# ds_doy = ren_ds.copy(deep=True)
# ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
# ds_doy['year'] = ds_doy.time.dt.year
# ds_doy = ds_doy.assign_coords(
#     {"dayofyear":ds_doy.time.dt.dayofyear,
#     "year":ds_doy.time.dt.year}
# )
# # reshape time dimension
# ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()

# drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()

# # reshape back into daily timeseries
# drought_ds = drought_ds.stack(time=['year','dayofyear'])
# drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)

# drought_ds = drought_ds.load()

# # Create binary drought mask for entire time series
# # 1 where drought_ds < 0 (below threshold), 0 otherwise
# drought_mask = xr.where(drought_ds < 0, 1, 0)
# drought_mask.name = "drought_mask"

# # save into the drought mask folder

# drought_mask = drought_mask.load()

# # Save drought mask to file
# mask_output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_drought_mask_only_full_time.nc"

# # Add metadata to drought mask
# drought_mask.attrs['resource'] = resource
# drought_mask.attrs['module'] = module
# drought_mask.attrs['domain'] = domain
# drought_mask.attrs['variable'] = variable
# drought_mask.attrs['simulation'] = simulation
# drought_mask.attrs['reference_gwl'] = float(reference_gwl)
# drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

# # Save with appropriate encoding
# encoding = {'drought_mask': {'dtype': 'int32', '_FillValue': -999}}
# drought_mask.to_netcdf(mask_output_file, encoding=encoding, format='NETCDF4')

# print(f"Saved drought mask to: {mask_output_file}")
# print(f"File size: {os.path.getsize(mask_output_file) / 1e6:.2f} MB")

: 

In [4]:
def create_drought_mask_gwl(resource, module, simulation, reference_gwl, mask_gwl):
    domain = "d03"
    variable = "cf"
    frequency = "day"

    print(f"Calculating drought mask for GWL: {mask_gwl}°C")
    # Get bounds for reference GWL period
    WRF_sim_name = sim_name_dict[simulation]
    model = WRF_sim_name.split("_")[1]
    ensemble_member = WRF_sim_name.split("_")[2]
    ref_start_year, ref_end_year = get_gwl_crossing_period(model, ensemble_member, mask_gwl)


    threshold_file = f"../data/thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
    drought_threshold_ds = xr.open_dataset(threshold_file)

    # clear encoding to avoid Zarr v3 codec issues
    for var in drought_threshold_ds.data_vars:
        drought_threshold_ds[var].encoding = {}

    ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
    ren_ds = ren_ds.convert_calendar("noleap")
    ren_ds = ren_ds.sel(time=slice(f"{ref_start_year}-01-01",f"{ref_end_year}-12-31"))

    ## Subtract the threshold from the ren data to get drought values
    # reshape array
    ds_doy = ren_ds.copy(deep=True)
    ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
    ds_doy['year'] = ds_doy.time.dt.year
    ds_doy = ds_doy.assign_coords(
        {"dayofyear":ds_doy.time.dt.dayofyear,
        "year":ds_doy.time.dt.year})
    # reshape time dimension
    ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()
    drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()
    # reshape back into daily timeseries
    drought_ds = drought_ds.stack(time=['year','dayofyear'])
    drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)
    drought_ds = drought_ds.load()


    # Create binary drought mask for entire time series
    # 1 where drought_ds < 0 (below threshold), 0 otherwise
    drought_mask = xr.where(drought_ds < 0, 1, 0)
    drought_mask.name = "drought_mask"

    # save into the drought mask folder
    drought_mask = drought_mask.load()

    # Save drought mask to file
    mask_output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwl{mask_gwl}_drought_mask_only.zarr"

    # Add metadata to drought mask
    drought_mask.attrs['resource'] = resource
    drought_mask.attrs['module'] = module
    drought_mask.attrs['domain'] = domain
    drought_mask.attrs['variable'] = variable
    drought_mask.attrs['simulation'] = simulation
    drought_mask.attrs['reference_gwl'] = float(reference_gwl)
    drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

    # Clear encoding from data variable and all coordinates to avoid codec conflicts
    drought_mask.encoding = {}
    for coord in drought_mask.coords:
        drought_mask.coords[coord].encoding = {}
    
    # Save with Zarr v3 - let xarray choose v3-compatible codecs
    # Only specify dtype, let compression use v3 defaults
    encoding = {'drought_mask': {'dtype': 'int32'}}
    drought_mask.to_zarr(mask_output_file, encoding=encoding, mode='w', consolidated=True)

In [ ]:
resource = "windpower"
module = "offshore"
reference_gwl = 0.8
mask_gwl = 2.0
simulation = "miroc6"

create_drought_mask_gwl(resource, module, simulation, reference_gwl, mask_gwl)

Calculating drought mask for GWL: 2.0°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [6]:
resource = "windpower"
module = "offshore"

reference_gwl = 0.8

for simulation in ["mpi-esm1-2-hr","taiesm1", 'ec-earth3',"miroc6"]:
    for mask_gwl in [0.8,2.0,]:
        print(f"Processing simulation: {simulation}, mask GWL: {mask_gwl}°C")
        create_drought_mask_gwl(resource, module, simulation, reference_gwl, mask_gwl)

Processing simulation: mpi-esm1-2-hr, mask GWL: 0.8°C
Calculating drought mask for GWL: 0.8°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: mpi-esm1-2-hr, mask GWL: 2.0°C
Calculating drought mask for GWL: 2.0°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: taiesm1, mask GWL: 0.8°C
Calculating drought mask for GWL: 0.8°C


...

/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: miroc6, mask GWL: 0.8°C
Calculating drought mask for GWL: 0.8°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Processing simulation: miroc6, mask GWL: 2.0°C
Calculating drought mask for GWL: 2.0°C


/Users/work/miniconda3/envs/renewables-analysis/lib/python3.12/site-packages/zarr/api/asynchronous.py:244: ZarrUserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


In [4]:
# calculate drought mask for a single GWL period

mask_gwl = reference_gwl


for mask_gwl in [reference_gwl, future_gwl]:
    print(f"Calculating drought mask for GWL: {mask_gwl}°C")
    # Get bounds for reference GWL period
    WRF_sim_name = sim_name_dict[simulation]
    model = WRF_sim_name.split("_")[1]
    ensemble_member = WRF_sim_name.split("_")[2]
    ref_start_year, ref_end_year = get_gwl_crossing_period(model, ensemble_member, mask_gwl)

    domain = "d02"
    variable = "cf"
    frequency = "day"

    threshold_file = f"../data/thresholds/{resource}_{module}_{domain}_{variable}_{simulation}_gwlref{reference_gwl}_10th_pctile.nc"
    drought_threshold_ds = xr.open_dataset(threshold_file)
    ren_ds = get_ren_data_concat(resource, module, domain, variable, frequency, simulation)
    ren_ds = ren_ds.convert_calendar("noleap")


    ren_ds = ren_ds.sel(time=slice(f"{ref_start_year}-01-01",f"{ref_end_year}-12-31"))

    # reshape array
    ds_doy = ren_ds.copy(deep=True)
    ds_doy['dayofyear'] = ds_doy.time.dt.dayofyear
    ds_doy['year'] = ds_doy.time.dt.year
    ds_doy = ds_doy.assign_coords(
        {"dayofyear":ds_doy.time.dt.dayofyear,
        "year":ds_doy.time.dt.year}
    )
    # reshape time dimension
    ds_doy = ds_doy.drop_vars("time").set_index(time=['dayofyear','year']).unstack()

    drought_ds = (ds_doy - drought_threshold_ds.reference_gen).load()

    # reshape back into daily timeseries
    drought_ds = drought_ds.stack(time=['year','dayofyear'])
    drought_ds = drought_ds.reset_index("time").assign_coords(time=ren_ds.time)

    drought_ds = drought_ds.load()

    # Create binary drought mask for entire time series
    # 1 where drought_ds < 0 (below threshold), 0 otherwise
    drought_mask = xr.where(drought_ds < 0, 1, 0)
    drought_mask.name = "drought_mask"

    # save into the drought mask folder

    drought_mask = drought_mask.load()

    # Save drought mask to file
    mask_output_file = f"../data/drought_masks/{resource}_{module}_{domain}_{variable}_{simulation}_gwl{mask_gwl}_drought_mask_only.nc"

    # Add metadata to drought mask
    drought_mask.attrs['resource'] = resource
    drought_mask.attrs['module'] = module
    drought_mask.attrs['domain'] = domain
    drought_mask.attrs['variable'] = variable
    drought_mask.attrs['simulation'] = simulation
    drought_mask.attrs['reference_gwl'] = float(reference_gwl)
    drought_mask.attrs['description'] = 'Binary drought mask: 1 = drought (below threshold), 0 = no drought'

    # Save with appropriate encoding
    encoding = {'drought_mask': {'dtype': 'int32', '_FillValue': -999}}
    drought_mask.to_netcdf(mask_output_file, encoding=encoding, format='NETCDF4')

    print(f"Saved drought mask to: {mask_output_file}")
    print(f"File size: {os.path.getsize(mask_output_file) / 1e6:.2f} MB")

Calculating drought mask for GWL: 0.8°C
Saved drought mask to: ../data/drought_masks/windpower_onshore_d02_cf_mpi-esm1-2-hr_gwl0.8_drought_mask_only.nc
File size: 3505.24 MB
Calculating drought mask for GWL: 2.0°C
Saved drought mask to: ../data/drought_masks/windpower_onshore_d02_cf_mpi-esm1-2-hr_gwl2.0_drought_mask_only.nc
File size: 3505.24 MB


## Interactive spatial visualization of drought mask

In [ ]:
# from ipywidgets import interact, IntSlider
# import matplotlib.pyplot as plt

# def plot_drought_mask_timestep(time_idx):
#     """Plot the spatial drought mask for a given time index"""
#     fig, ax = plt.subplots(figsize=(10, 8))
    
#     # Get the drought mask at this time step
#     mask_slice = drought_mask.isel(time=time_idx)
    
#     # Plot
#     im = ax.pcolormesh(
#         mask_slice.x, 
#         mask_slice.y, 
#         mask_slice.values,
#         cmap='RdYlBu_r',
#         vmin=0,
#         vmax=1,
#         shading='auto'
#     )
    
#     # Get the time value
#     time_val = drought_mask.time.values[time_idx]
    
#     ax.set_title(f'Drought Mask - Time: {time_val}', fontsize=14)
#     ax.set_xlabel('X coordinate')
#     ax.set_ylabel('Y coordinate')
#     ax.set_aspect('equal')
    
#     # Add colorbar
#     cbar = plt.colorbar(im, ax=ax)
#     cbar.set_label('Drought (1) / No Drought (0)', rotation=270, labelpad=20)
    
#     plt.tight_layout()
#     plt.show()

# # Create interactive slider
# n_times = len(drought_mask.time)
# time_slider = IntSlider(
#     min=0, 
#     max=n_times-1, 
#     step=1, 
#     value=0,
#     description='Time step:',
#     continuous_update=False
# )

# interact(plot_drought_mask_timestep, time_idx=time_slider)

interactive(children=(IntSlider(value=0, continuous_update=False, description='Time step:', max=43069), Output…

<function __main__.plot_drought_mask_timestep(time_idx)>